In [1]:
import pandas as pd
import json
from glob import glob
from tqdm import tqdm 
import contractions

In [2]:
path_to_pos_reviews = "../data/raw/imdb/train/pos"
path_to_neg_reviews = "../data/raw/imdb/train/neg"
pos_reviews = glob(path_to_pos_reviews+"/*txt")
neg_reviews = glob(path_to_neg_reviews+"/*txt")

In [3]:
all_entries = []
all_files = pos_reviews + neg_reviews
for file in tqdm(all_files):
    f_name = file.split("/")[-1]
    f_name = f_name.replace(".txt", "")
    _id,u_ratings = map(int, f_name.split("_"))
    with open(file) as review:
        u_review = review.read()
    entry = {}
    entry["id"] = _id
    entry["ratings"] = 0 if u_ratings < 5 else 1 # postive: 1, negative: 0
    entry["review"] = u_review
    all_entries.append(entry)
    pass

review_df = pd.DataFrame(all_entries)
review_df.count()

100%|██████████| 25000/25000 [00:08<00:00, 2871.68it/s]


id         25000
ratings    25000
review     25000
dtype: int64

In [4]:
review_df.head()

,id,ratings,review
0,4715,1,For a movie that gets no respect there sure ar...
1,12390,1,Bizarre horror movie filled with famous faces ...
2,8329,1,"A solid, if unremarkable film. Matthau, as Ein..."
3,9063,1,It's a strange feeling to sit alone in a theat...
4,3092,1,"You probably all already know this by now, but..."


## Data Preprocessing: Cleaning the reviews

 - Expanding Contractions
 - Language Detection
 - Tokenization
 - Converting all Characters to Lowercase
 - Removing Punctuations
 - Removing Stopwords
 - Parts of Speech Tagging
 - Lemmatization

### Contrations expansion

In [5]:
review_df["c_review"] = review_df["review"].apply(lambda x: [
    contractions.fix(word) for word in x.split()])
review_df["exp_review"] = review_df["c_review"].apply(lambda x: ' '.join(x))
review_df = review_df.drop(columns=["c_review"])
review_df.head()

,id,ratings,review,exp_review
0,4715,1,For a movie that gets no respect there sure ar...,For a movie that gets no respect there sure ar...
1,12390,1,Bizarre horror movie filled with famous faces ...,Bizarre horror movie filled with famous faces ...
2,8329,1,"A solid, if unremarkable film. Matthau, as Ein...","A solid, if unremarkable film. Matthau, as Ein..."
3,9063,1,It's a strange feeling to sit alone in a theat...,it is a strange feeling to sit alone in a thea...
4,3092,1,"You probably all already know this by now, but...","You probably all already know this by now, but..."


### Language Detection

In [9]:
import fasttext

pretrained_model = "lid.176.bin" 
model = fasttext.load_model(pretrained_model)

langs = []
for sent in tqdm(review_df['exp_review']):
    lang = model.predict(sent)[0]
    langs.append(str(lang)[11:13])
    pass

review_df['langs'] = langs

ValueError: lid.176.bin cannot be opened for loading!

### Tokenisation

### Converting all Characters to Lowercase

### Removing Punctuations

### Removing StopWords

### Parts of Speech Tagging

### Lemmatization